<a href="https://colab.research.google.com/github/JonMyles494/My-First-Repo/blob/main/Problem_7_1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The 6.00 Word Game

import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

# -----------------------------------
# Helper code
# (you don't need to understand this helper code)

WORDLIST_FILENAME = "words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.

    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq


# (end of helper code)
# -----------------------------------

#
# Problem #1: Scoring a word
#
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    if word == "": # Edge case: in case string is empty return zero
      return 0

    # Calculate base score of word
    score = sum(SCRABBLE_LETTER_VALUES[letter] for letter in word)

    # Multiply by the length of the word
    score *= len(word)

    # Apply the bonus if all of the letters in the hand were used
    if len(word) == n:
      score += 50
      #print("Bouns applied")
    return score



#
# Problem #2: Make sure you understand how this function works and what it does!
#
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()                             # print an empty line

#
# Problem #2: Make sure you understand how this function works and what it does!
#
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3

    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1

    for i in range(numVowels, n):
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1

    return hand

#
# Problem #2: Update a hand by removing letters
#
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it.

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)
    returns: dictionary (string -> int)
    """
  # Make a copy of the original hand for mutability
    new_hand = hand.copy()

    # Update the hand by removing letters in the word
    for letter in word:
      if letter in word:
        new_hand[letter] -= 1  # Subtract 1 from the letter's count
        if new_hand[letter] == 0:
          del new_hand[letter]  # Remove the letter if count reaches zero
    return new_hand



#
# Problem #3: Test word validity
#
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.

    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    # Check to see if the is in the wordList
    if word not in wordList:
      return False

    # Create new copy of the hand to avoid changing it
    hand_copy = hand.copy()

    # Check to see if the word can be formed formed the hand
    for letter in word:
      if hand_copy.get(letter, 0) == 0:   # If the letter is used or not available
        return False
      else:
        hand_copy[letter] -= 1  # Use up an occurrence of the letter
    return True

#
# Problem #4: Calculate a hand
#

def calculateHandlen(hand):
    """
    Returns the length (number of letters) in the current hand.

    hand: dictionary (string-> int)
    returns: integer
    """
    return sum(hand.values())

#
# Problem #5: Playing a hand
#

def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".")
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)

    """
    # BEGIN PSEUDOCODE <-- Remove this comment when you code this function; do your coding within the pseudocode (leaving those comments in-place!)
    # Keep track of the total score
    total_score = 0
    # As long as there are still letters left in the hand:
    while calculateHandlen(hand) > 0:
        # Display the hand
        print("Current Hand: ", end="")
        displayHand(hand)
        # Ask user for input
        word = input("Enter the word, or a '.' to indicate that you are finished: ")
        # If the input is a single period:
        if word == ".":
            # End the game (break out of the loop)
            print(f"Goodbye! Total Score: {total_score} points")
            break
        # Otherwise (the input is not a single period):
        else:
            # If the word is not valid:
            if not isValidWord(word, hand, wordList):
                # Reject invalid word (print a message followed by a blank line)
                print("Invalid word, please try again.\n")
            # Otherwise (the word is valid):
            else:
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line
                word_score = getWordScore(word, n)
                total_score += word_score
                print(f'"{word}" earned {word_score} points. Total: {total_score} points.\n')

                # Update the hand
                hand = updateHand(hand, word)

    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    if calculateHandlen(hand) == 0:
      print(f"Ran out of letters. Total Score: {total_score} points.")

#
# Problem #6: Playing a game
#

def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.

    2) When done playing the hand, repeat from step 1
    """
    # TO DO ... <-- Remove this comment when you code this function
    print("playGame not yet implemented.") # <-- Remove this line when you code the function




#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)







In [ ]:
#from ps4a import *
import time


#
#
# Computer chooses a word
#
#
def compChooseWord(hand, wordList, n):
    """
    Given a hand and a wordList, find the word that gives
    the maximum value score, and return it.

    This word should be calculated by considering all the words
    in the wordList.

    If no words in the wordList can be made from the hand, return None.

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)

    returns: string or None
    """
    # Create a new variable to store the maximum score seen so far (initially 0)
    bestScore = 0
    # Create a new variable to store the best word seen so far (initially None)
    bestWord = None
    # For each word in the wordList
    for word in wordList:
        # If you can construct the word from your hand
        if isValidWord(word, hand, wordList):
            # find out how much making that word is worth
            score = getWordScore(word, n)
            # If the score for that word is higher than your best score
            if (score > bestScore):
                # update your best score, and best word accordingly
                bestScore = score
                bestWord = word
    # return the best word you found.
    return bestWord

#
# Computer plays a hand
#
def compPlayHand(hand, wordList, n):
    """
    Allows the computer to play the given hand, following the same procedure
    as playHand, except instead of the user choosing a word, the computer
    chooses it.

    1) The hand is displayed.
    2) The computer chooses a word.
    3) After every valid word: the word and the score for that word is
    displayed, the remaining letters in the hand are displayed, and the
    computer chooses another word.
    4)  The sum of the word scores is displayed when the hand finishes.
    5)  The hand finishes when the computer has exhausted its possible
    choices (i.e. compChooseWord returns None).

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    """
    # Keep track of the total score
    totalScore = 0
    # As long as there are still letters left in the hand:
    while (calculateHandlen(hand) > 0) :
        # Display the hand
        print("Current Hand: ", end=' ')
        displayHand(hand)
        # computer's word
        word = compChooseWord(hand, wordList, n)
        # If the input is a single period:
        if word == None:
            # End the game (break out of the loop)
            break

        # Otherwise (the input is not a single period):
        else :
            # If the word is not valid:
            if (not isValidWord(word, hand, wordList)) :
                print('This is a terrible error! I need to check my own code!')
                break
            # Otherwise (the word is valid):
            else :
                # Tell the user how many points the word earned, and the updated total score
                score = getWordScore(word, n)
                totalScore += score
                print('"' + word + '" earned ' + str(score) + ' points. Total: ' + str(totalScore) + ' points')
                # Update hand and show the updated hand to the user
                hand = updateHand(hand, word)
                print()
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Total score: ' + str(totalScore) + ' points.')


#
# Problem #6: Playing a game
#
#
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.

        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    last_hand = None  # Keep track of the last hand

    while True:
      # Prompt the user for input
      choice = input("Enter n to deal a new hand, r to replace the last hand, or e to end the game: ")
      # User exits the game
      if choice == 'e':
        print("Goodbye!")
        break
      # if user chooses to play a new hand
      if choice == 'n':
        hand = dealHand(HAND_SIZE) # Deal a new hand
        last_hand = hand    # Update the last_hand
        playHand(hand, wordList, HAND_SIZE)
      # player wants to replay the last hand
      elif choice == 'r':
        if last_hand is None:    # Idicates there is not a hand to play
          print("You have bot yet played a hand, please play a hand first!")
        else:
          playHand(last_hand, wordList, HAND_SIZE)   # Replay last hand
      # Invalid input case
      else:
        print("Invalid Command.")
        continue
    # Computer play choice
#from ps4a import *
import time


#
#
# Computer chooses a word
#
#
def compChooseWord(hand, wordList, n):
    """
    Given a hand and a wordList, find the word that gives
    the maximum value score, and return it.

    This word should be calculated by considering all the words
    in the wordList.

    If no words in the wordList can be made from the hand, return None.

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)

    returns: string or None
    """
    # Create a new variable to store the maximum score seen so far (initially 0)
    bestScore = 0
    # Create a new variable to store the best word seen so far (initially None)
    bestWord = None
    # For each word in the wordList
    for word in wordList:
        # If you can construct the word from your hand
        if isValidWord(word, hand, wordList):
            # find out how much making that word is worth
            score = getWordScore(word, n)
            # If the score for that word is higher than your best score
            if (score > bestScore):
                # update your best score, and best word accordingly
                bestScore = score
                bestWord = word
    # return the best word you found.
    return bestWord

#
# Computer plays a hand
#
def compPlayHand(hand, wordList, n):
    """
    Allows the computer to play the given hand, following the same procedure
    as playHand, except instead of the user choosing a word, the computer
    chooses it.

    1) The hand is displayed.
    2) The computer chooses a word.
    3) After every valid word: the word and the score for that word is
    displayed, the remaining letters in the hand are displayed, and the
    computer chooses another word.
    4)  The sum of the word scores is displayed when the hand finishes.
    5)  The hand finishes when the computer has exhausted its possible
    choices (i.e. compChooseWord returns None).

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    """
    # Keep track of the total score
    totalScore = 0
    # As long as there are still letters left in the hand:
    while (calculateHandlen(hand) > 0) :
        # Display the hand
        print("Current Hand: ", end=' ')
        displayHand(hand)
        # computer's word
        word = compChooseWord(hand, wordList, n)
        # If the input is a single period:
        if word == None:
            # End the game (break out of the loop)
            break

        # Otherwise (the input is not a single period):
        else :
            # If the word is not valid:
            if (not isValidWord(word, hand, wordList)) :
                print('This is a terrible error! I need to check my own code!')
                break
            # Otherwise (the word is valid):
            else :
                # Tell the user how many points the word earned, and the updated total score
                score = getWordScore(word, n)
                totalScore += score
                print('"' + word + '" earned ' + str(score) + ' points. Total: ' + str(totalScore) + ' points')
                # Update hand and show the updated hand to the user
                hand = updateHand(hand, word)
                print()
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Total score: ' + str(totalScore) + ' points.')


#
# Problem #6: Playing a game
#
#
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.

        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    last_hand = None  # Keep track of the last hand

    while True:
      # Prompt the user for input
      choice = input("Enter n to deal a new hand, r to replace the last hand, or e to end the game: ")
      # User exits the game
      if choice == 'e':
        print("Goodbye!")
        break
      # if user chooses to play a new hand
      elif choice == 'n':
        hand = dealHand(HAND_SIZE) # Deal a new hand
        last_hand = hand    # Update the last_hand
        #playHand(hand, wordList, HAND_SIZE)
      # player wants to replay the last hand
      elif choice == 'r':
        if last_hand is None:    # Idicates there is not a hand to play
          print("You have not yet played a hand, please play a hand first!")

          continue
        else:
          playHand(last_hand, wordList, HAND_SIZE)   # Replay last hand
      # Invalid input case
      else:
        print("Invalid Command.")
        continue

    # Computer play choice
      while True:
        player_choice = input("Enter 'u' to play the hand or 'c' to have the computer play the hand: ")   # Prompt user to play or let the computer play
        # Test to see what user selected
        if player_choice == 'u':
          playHand(hand, wordList, HAND_SIZE)
          break
        elif player_choice == 'c':
          compPlayHand(hand, wordList, HAND_SIZE)
          break
        else:
          print("Invalid choice!")
#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)



#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)




In [11]:

# Test cases for compPlayHand function

print("\nTesting compPlayHand function:")

# Test case 1 for compPlayHand
hand1 = {'a': 1, 'p': 2, 's': 1, 'e': 1, 'l': 1}
compPlayHand(hand1, wordList, 6)  # Expected output: "appels" earned 110 points. Total: 110 points.

# Test case 2 for compPlayHand
hand2 = {'a': 2, 'c': 1, 'b': 1, 't': 1}
compPlayHand(hand2, wordList, 5)  # Expected output: "acta" earned 24 points. Total: 24 points.

# Test case 3 for compPlayHand
hand3 = {'a': 2, 'e': 2, 'i': 2, 'm': 2, 'n': 2, 't': 2}

compPlayHand(hand3, wordList, 12)  # Expected output sequence:
                                   # "immanent" earned 96 points. Total: 96 points.
                                   # "ait" earned 9 points. Total: 105 points.


Testing compPlayHand function:
Current Hand:  a p p s e l 
Bouns applied
Bouns applied
Bouns applied
"appels" earned 110 points. Total: 110 points

Total score: 110 points.
Current Hand:  a a c b t 
"acta" earned 24 points. Total: 24 points

Current Hand:  b 
Total score: 24 points.
Current Hand:  a a e e i i m m n n t t 
"immanent" earned 96 points. Total: 96 points

Current Hand:  a e i t 
"ait" earned 9 points. Total: 105 points

Current Hand:  e 
Total score: 105 points.



.
.
.
.
.
.
.
.
.
.

In [90]:
# Test playHand function

# Sample data for testing
n = 7
hand = {'a': 1, 'q': 1, 'l': 2, 'm': 1, 'u': 1, 'i': 1}
wordList = ['alum', 'quail', 'lima', 'aim', 'mail', 'ail', 'lam', 'a', 'la', 'qi']

# Run playHand with a predefined hand and word list
print("Testing playHand function with sample hand and wordList...\n")
playHand(hand, wordList, n)


Testing playHand function with sample hand and wordList...

Current Hand: a q l l m u i 
Enter the word, or a '.' to indicate that you are finished: quail
"quail" earned 70 points. Total: 70 points.

Current Hand: l m 
Enter the word, or a '.' to indicate that you are finished: .
Goodbye! Total Score: 70 points


In [ ]:
# isValidWord function test cases

# Sample word list and a hand
wordList = ['hello', 'hell', 'he', 'oat', 'oath', 'quail']
hand = {'h': 2, 'e': 1, 'l': 2, 'o': 1, 'a': 1, 't': 1, 'q': 1, 'u': 1, 'i': 1}

# Validate 'hello' from the hand
print(f"Is 'hello' valid? (Expected: True): {isValidWord('hello', hand, wordList)}")

# Validate 'quail' from the hand
print(f"Is 'quail' valid? (Expected: True): {isValidWord('quail', hand, wordList)}")

# Validate 'quill' (not enough 'l's in the hand)
print(f"Is 'quill' valid? (Expected: False): {isValidWord('quill', hand, wordList)}")

# Empty string is not a valid word
print(f"Is '' valid? (Expected: False): {isValidWord('', hand, wordList)}")

# Repeated hand use for multiple valid words
print(f"Is 'hell' valid? (Expected: True): {isValidWord('hell', hand, wordList)}")
print(f"Is 'oath' valid? (Expected: True): {isValidWord('oath', hand, wordList)}")
print(f"Is 'oat' valid? (Expected: True): {isValidWord('oat', hand, wordList)}")

# Test playHand function

# Sample data for testing
sample_hand = {'a': 1, 'q': 1, 'l': 2, 'm': 1, 'u': 1, 'i': 1}
wordList = ['alum', 'quail', 'lima', 'aim', 'mail', 'ail', 'lam', 'a', 'la', 'qi']

# Run playHand with a predefined hand and word list
print("Testing playHand function with sample hand and wordList...\n")
playHand(sample_hand, wordList, HAND_SIZE=7)


# Test calculateHandlen function
print(f"Hand length (Expected: 10): {calculateHandlen(hand)}")

# Test cases for compChooseWord function

print("Testing compChooseWord function:")

# The sample word list (wordList) for testing
wordList = ['appels', 'acta', 'immanent', 'ait']

# Word with hand {'a': 1, 'p': 2, 's': 1, 'e': 1, 'l': 1}
hand1 = {'a': 1, 'p': 2, 's': 1, 'e': 1, 'l': 1}
print(f"Best word for hand {hand1}: {compChooseWord(hand1, wordList, 6)}")  # Expected output: "appels"

# TWord with hand {'a': 2, 'c': 1, 'b': 1, 't': 1}
hand2 = {'a': 2, 'c': 1, 'b': 1, 't': 1}
print(f"Best word for hand {hand2}: {compChooseWord(hand2, wordList, 5)}")  # Expected output: "acta"

# Word with hand {'a': 2, 'e': 2, 'i': 2, 'm': 2, 'n': 2, 't': 2}
hand3 = {'a': 2, 'e': 2, 'i': 2, 'm': 2, 'n': 2, 't': 2}
print(f"Best word for hand {hand3}: {compChooseWord(hand3, wordList, 12)}")  # Expected output: "immanent"

# TWord with hand {'x': 2, 'z': 2, 'q': 2, 'n': 2, 't': 2}
hand4 = {'x': 2, 'z': 2, 'q': 2, 'n': 2, 't': 2}
print(f"Best word for hand {hand4}: {compChooseWord(hand4, wordList, 12)}")  # Expected output: None



# Module test for getWordScore(), updateHand(), & isValidWord()
!python3 test_ps4a.py
